## Retrieving Data

Download from Kaggle & Optional Upload to Minio Local deployment

In [ ]:
# Start by downloading the necessary data. You will need to have setup your Kaggle API access
# https://www.kaggle.com/docs/api#interacting-with-datasets
# Alternatively, download the data through the web ui and put it into the root folder into ./data
import sys
sys.path.append("/home/thomas/workspace/mlops_sea_animal_classification/")
!pdm run kaggle datasets download -d vencerlanz09/sea-animals-image-dataste
!mkdir ../data && unzip sea-animals-image-dataste -d ../data && rm -f sea-animals-image-dataste.zip

In [ ]:
# Use this if you want to store data on your minio cluster
import glob
import os
from minio import Minio

client = Minio("localhost:9000", "minio", "minio123", secure=False)

def upload_local_directory_to_minio(local_path, bucket_name, minio_path):
    
    assert os.path.isdir(local_path)

    for local_file in glob.glob(local_path + '/**'):
        local_file = local_file.replace(os.sep, "/") # Replace \ with / on Windows
        if not os.path.isfile(local_file):
            upload_local_directory_to_minio(
                local_file, bucket_name, minio_path + "/" + os.path.basename(local_file))
        else:
            remote_path = os.path.join(
                minio_path, local_file[1 + len(local_path):])
            remote_path = remote_path.replace(
                os.sep, "/")  # Replace \ with / on Windows
            client.fput_object(bucket_name, remote_path, local_file)

In [ ]:
local_path = "../data"
bucket_name = "sea-animals"
minio_path = "data"
upload_local_directory_to_minio(local_path, bucket_name, minio_path)

## Download Data from Minio

In [ ]:
bucket_name = "sea-animals"

for item in client.list_objects(bucket_name,recursive=True):
    client.fget_object(bucket_name, item.object_name, item.object_name)